In [300]:
#import some libraries
import time
import requests,re
from bs4 import BeautifulSoup
import json
import pymysql as pymysql
import os
import pandas as pd
from datetime import datetime

In [301]:
def get_player_page_urls(url):    
    #request the different URLS based on function input
    url = requests.get(url,headers={'user-agent':'Mozilla/5.0'})
    #pull the HTML code
    soup = BeautifulSoup(url.content, "html.parser")
    #Find the table we want to scrap
    div = soup.find('div', id='all_players_standard_pitching')
    #find the contents we want to scrap
    bdiv = bytearray(str(div.contents),'utf-8')
    #pull that contents into a soup object 
    soup2 = BeautifulSoup(bdiv, "html.parser")
    tbody = soup2.find('tbody')
    #find all the rows we want
    rows = tbody.find_all('tr')
    
    #get all the urls for the players'personal info page 
    player_link=list()
    player_name=list()
    i=0
    for row in rows:
        i+=1
        try:
            str_row=str(row)
            num_row = row.find('th').text
            name = row.find('a').text
            link = re.findall('<a.+?href="(/players/.+?)">',str_row)
            if i % 100 == 0:
                print('saved',i,'page urls','now saving player:',name,"...")
            player_link.append(link)
            player_name.append(name)
        except:
            continue
    return player_link,player_name

In [306]:
years = ["2015","2016","2017","2018"]
all_player_link_list = []
for year in years:
    print('Saving year',year,'info...')
    url = "https://www.baseball-reference.com/leagues/MLB/"+year+"-standard-pitching.shtml"
    #begin scraping 
    #call the functions and find all player urls 
    player_link_list,player_name_list = get_player_page_urls(url)
    all_player_link_list.append(player_link_list)
print('Finish saving!')

Saving year 2015 info...
saved 100 page urls now saving player: Madison Bumgarner ...
saved 200 page urls now saving player: Jose De Paula ...
saved 300 page urls now saving player: Gonzalez Germen ...
saved 400 page urls now saving player: Tommy Hunter ...
saved 500 page urls now saving player: Javier Lopez ...
saved 600 page urls now saving player: Jimmy Nelson ...
saved 700 page urls now saving player: Robbie Ray ...
saved 800 page urls now saving player: Joakim Soria ...
saved 900 page urls now saving player: Asher Wojciechowski ...
Saving year 2016 info...
saved 100 page urls now saving player: Matt Bush ...
saved 200 page urls now saving player: Odrisamer Despaigne ...
saved 300 page urls now saving player: Koda Glover ...
saved 400 page urls now saving player: Hisashi Iwakuma ...
saved 500 page urls now saving player: Matt Magill ...
saved 600 page urls now saving player: Justin Nicolino ...
saved 700 page urls now saving player: Anthony Ranaudo ...
saved 800 page urls now savin

In [319]:
#save the urls into txt files
f = open('all_player_link_list.txt','w')
for link_list in all_player_link_list:
    for i in range(len(link_list)):
        f.write(link_list[i][0])
        f.write('\n')
f.close()

In [415]:
# #CHECK REGEX
# r=requests.get("https://www.baseball-reference.com/players/a/almonmi01.shtml",{'user-agent':'Mozilla/5.0'})
# soup = BeautifulSoup(r.content, 'html.parser')
# #Find the info of items and save
# info_list=[]
# #Find the info of items, convert to suitable data type and save
# #'Name','Height','Weight','Birthday','Age','Debut Age','Birthplace','High School','School','Rookie Status','2020 Contract Status',
#           #,'Arb Eligible','Free Agent','Full Name'
# info_text = soup.find('div',itemtype="https://schema.org/Person").text.replace(u'\n',u' ').replace(u'\xa0',u' ').replace(u'\t',u' ')

# #=========== Find Name =======================================
# try:
#     pitcher_name=soup.find('div','players').find('h1').text
# except:
#     pitcher_name= 'N/A'


# #=========== Find Height =======================================
# try:
    
#     pitcher_height=soup.find('span',itemprop='height').text.replace('-','.')

# except:
#     pitcher_height= 'NULL'


# #=========== Find Weight=======================================
# try:
#     pitcher_weight=soup.find('span',itemprop='weight').text.strip('lb')

# except:
#     pitcher_weight= 'NULL'

# #=========== Find birthday=======================================
# try:
#     birthday=re.findall(r'data-birth=(.*)id',str(soup.find('span',itemprop='birthDate')))[0].replace('"','')
# except:
#     birthday= 'NULL'


# #=========== Find Age=======================================
# try:
#     now = datetime.now()
#     age = str(int(now.year) - int(birthday[:4]))
# except:
#     age= 'NULL'    

# #=========== Find Debut Age=======================================
# try:
#     debut_age=re.findall(r'Age.?([0-9]{2}.?[0-9]{0,4}d)',info_text)[0]
# except:
#     debut_age= 'NULL'
    
# #=========== Find Birthplace =======================================
# try:
#     birthplace =soup.find('span',itemprop='birthPlace').text.strip().replace(u'in',u'').replace(u'\n',u'')
# except:
#     birthplace= 'NULL'

# #=========== ***** Find High School=======================================
# try:
#     high_school=re.findall(r'High School: ([a-zA-Z_0-9 ]*)',info_text)[0].replace(u'\n',u'').replace(u'\t',u'')
# except:
#     high_school= 'NULL'

# #=========== ***** Find School=======================================
# try:
#     school=re.findall(r'High School: .* School: ([a-zA-Z_0-9 ]*)',info_text)[0].replace(u'in',u'').replace(u'\n',u'').replace(u'\t',u'')
#     if school == high_school:
#             school = 'NULL'
# except:
#     school= 'NULL'

# #=========== Find Rookie Status=======================================
# try:
#     Rookie_Status=re.findall(r'Rookie Status: ([\-a-zA-Z_0-9 .]*)2020 Contract Status',info_text)[0].strip(' ')
# except:
#     Rookie_Status= 'NULL'

# # #=========== Find 2020 Contract Status=======================================
# try:
#     Contract_Status=re.findall(r'2020 Contract Status: ([\-a-zA-Z_0-9 .]*)Service Time',info_text)[0].strip(' ')
# except:
#     Contract_Status= 'NULL'
    

# # #=========== Find Arb Eligible=======================================
# try:
#     Arb_Eligible=re.findall(r'Arb Eligible: ([a-zA-Z_0-9 ]*)',info_text)[0].strip(' ').replace(u'in',u'').replace(u'\n',u'').replace(u'\t',u'')
# except:
#     Arb_Eligible= 'NULL'
    
    
# # #=========== Find Free Agent=======================================
# try:
#     Free_Agent=re.findall(r'Free Agent: ([a-zA-Z_0-9 ]*)Full Name:',info_text)[0].strip(' ').replace(u'in',u'').replace(u'\n',u'').replace(u'\t',u'')
# except:
#     Free_Agent= 'NULL'
    
# # #=========== Find Full Name=======================================
# try:
#     Full_Name=re.findall(r'Full Name:</strong>([a-zA-Z_0-9 .]*)',str(soup.find('div',itemtype="https://schema.org/Person")))[0].strip()
# except:
#     Full_Name= 'NULL'

    
# #save all info
# info_list=[pitcher_name,pitcher_height,pitcher_weight,birthday,age,birthplace,
#            high_school,school,Rookie_Status,Contract_Status,
#            Arb_Eligible,Free_Agent,Full_Name]

In [416]:
info_list

['Miguel Almonte',
 '6.1',
 '210',
 '1993-04-04 ',
 '27',
 ' Santiago, Domican Republic',
 'NULL',
 'NULL',
 'NULL',
 'NULL',
 'NULL',
 'NULL',
 'Miguel Emilio Almonte']

In [417]:
def get_player_info(url):
    r=requests.get(url,{'user-agent':'Mozilla/5.0'})
    soup = BeautifulSoup(r.content, 'html.parser')
    #Find the info of items and save
    info_list=[]
    #Find the info of items and save
    
    info_text = soup.find('div',itemtype="https://schema.org/Person").text.replace(u'\n',u' ').replace(u'\xa0',u' ').replace(u'\t',u' ')

    #=========== Find Name =======================================
    try:
        pitcher_name=soup.find('div','players').find('h1').text
    except:
        pitcher_name= 'N/A'


    #=========== Find Height =======================================
    try:

        pitcher_height=soup.find('span',itemprop='height').text.replace('-','.')

    except:
        pitcher_height= 'NULL'


    #=========== Find Weight=======================================
    try:
        pitcher_weight=soup.find('span',itemprop='weight').text.strip('lb')

    except:
        pitcher_weight= 'NULL'

    #=========== Find birthday=======================================
    try:
        birthday=re.findall(r'data-birth=(.*)id',str(soup.find('span',itemprop='birthDate')))[0].replace('"','')
    except:
        birthday= 'NULL'


    #=========== Find Age=======================================
    try:
        now = datetime.now()
        age = str(int(now.year) - int(birthday[:4]))
    except:
        age= 'NULL'    

    #=========== Find Debut Age=======================================
    try:
        debut_age=re.findall(r'Age.?([0-9]{2}.?[0-9]{0,4}d)',info_text)[0]
    except:
        debut_age= 'NULL'

    #=========== Find Birthplace =======================================
    try:
        birthplace =soup.find('span',itemprop='birthPlace').text.strip().replace(u'in',u'').replace(u'\n',u'')
    except:
        birthplace= 'NULL'

    #=========== ***** Find High School=======================================
    try:
        high_school=re.findall(r'High School: ([a-zA-Z_0-9 ]*)',info_text)[0].replace(u'\n',u'').replace(u'\t',u'')
    except:
        high_school= 'NULL'

    #=========== ***** Find School=======================================
    try:
        school=re.findall(r'High School: .* School: ([a-zA-Z_0-9 ]*)',info_text)[0].replace(u'in',u'').replace(u'\n',u'').replace(u'\t',u'')
        if school == high_school:
                school = 'NULL'
    except:
        school= 'NULL'

    #=========== Find Rookie Status=======================================
    try:
        Rookie_Status=re.findall(r'Rookie Status: ([\-a-zA-Z_0-9 .]*)2020 Contract Status',info_text)[0].strip(' ')
    except:
        Rookie_Status= 'NULL'

    # #=========== Find 2020 Contract Status=======================================
    try:
        Contract_Status=re.findall(r'2020 Contract Status: ([\-a-zA-Z_0-9 .]*)Service Time',info_text)[0].strip(' ')
    except:
        Contract_Status= 'NULL'


    # #=========== Find Arb Eligible=======================================
    try:
        Arb_Eligible=re.findall(r'Arb Eligible: ([a-zA-Z_0-9 ]*)',info_text)[0].strip(' ').replace(u'in',u'').replace(u'\n',u'').replace(u'\t',u'')
    except:
        Arb_Eligible= 'NULL'


    # #=========== Find Free Agent=======================================
    try:
        Free_Agent=re.findall(r'Free Agent: ([a-zA-Z_0-9 ]*)Full Name:',info_text)[0].strip(' ').replace(u'in',u'').replace(u'\n',u'').replace(u'\t',u'')
    except:
        Free_Agent= 'NULL'

    # #=========== Find Full Name=======================================
    try:
        Full_Name=re.findall(r'Full Name:</strong>([a-zA-Z_0-9 .]*)',str(soup.find('div',itemtype="https://schema.org/Person")))[0].strip()
    except:
        Full_Name= 'NULL'


    #save all info
    info_list=[pitcher_name,pitcher_height,pitcher_weight,birthday,age,birthplace,
               high_school,school,Rookie_Status,Contract_Status,
               Arb_Eligible,Free_Agent,Full_Name]

    return info_list

In [327]:
all_urls=[]
f = open("all_player_link_list.txt", "r")
for line in f:
    all_urls.append(line.strip('\n'))

In [418]:
all_info=[]
i=0
for link in all_urls:
    i+=1
    if i % 300 == 0:
        print('Saving',i,'player info...')
    url="https://www.baseball-reference.com"+link
    all_info.append(get_player_info(url))

Saving 300 player info...
Saving 600 player info...
Saving 900 player info...
Saving 1200 player info...
Saving 1500 player info...
Saving 1800 player info...
Saving 2100 player info...
Saving 2400 player info...
Saving 2700 player info...
Saving 3000 player info...
Saving 3300 player info...
Saving 3600 player info...


In [419]:
#label all the columns
columns = ['Name','Height/ft','Weight/lb','Birthday','Age','Birthplace','High School','School','Rookie Status','2020 Contract Status',
           'Arb Eligible','Free Agent','Full Name']
#makes these the column names in the dataframe
pitcher_info = pd.DataFrame(all_info,columns = columns)
pitcher_info.head(30)

,Name,Height/ft,Weight/lb,Birthday,Age,Birthplace,High School,School,Rookie Status,2020 Contract Status,Arb Eligible,Free Agent,Full Name
0,David Aardsma,6.3,215,1981-12-27,39,"Denver, CO",Cherry Creek HS,NULL,NULL,NULL,NULL,NULL,David Allan Aardsma
1,Fernando Abad,6.1,220,1985-12-17,35,"La Romana, Domican Republic",NULL,NULL,Exceeded rookie limits during 2011 season,Minor League Deal,NULL,NULL,Fernando Antonio Abad
2,A.J. Achter,6.5,215,1988-08-27,32,"Toledo, OH",Clay HS,Michigan State University,NULL,NULL,NULL,NULL,Adam Joseph Achter
3,Austin Adams,5.11,205,1986-08-19,34,"Montgomery, AL",Stanhope Elmore HS,NULL,Exceeded rookie limits during 2015 season,Pre-Arb Eligible,2022,NULL,Austin David Adams
4,Nathan Adcock,6.4,235,1988-02-25,32,"Elizabethtown, KY",North Hardin HS,NULL,NULL,NULL,NULL,NULL,Nathan Masler Adcock
5,Jeremy Affeldt,6.4,225,1979-06-06,41,"Phoenix, AZ",Northwest Christian HS,NULL,NULL,NULL,NULL,NULL,Jeremy David Affeldt
6,Andrew Albers,6.1,200,1985-10-06,35,"North Battleford, Canada",John Paul II HS,University of Kentucky,NULL,NULL,NULL,NULL,Andrew William Albers
7,Matt Albers,6.1,225,1983-01-20,37,"Houston, TX",William P,San Jacto College,Exceeded rookie limits during 2007 season,Free Agent,NULL,NULL,Matthew James Albers
8,Al Alburquerque,6.0,195,1986-06-10,34,"San Pedro de Macoris, Domican Republic",NULL,NULL,Exceeded rookie limits during 2011 season,3rd-Year Arb Eligible,2018,NULL,Alberto Jose Alburquerque
9,Scott Alexander,6.2,195,1989-07-10,31,"Santa Rosa, CA",Cardinal Newman HS,Sonoma State University,Exceeded rookie limits during 2017 season,NULL,2021,NULL,Scott Alain Alexander


In [420]:
pitcher_info.drop_duplicates(subset ="Name",keep = 'first', inplace = True)
pitcher_info.shape

(1328, 13)

In [421]:
pitcher_info.to_csv('pitcher_info.csv')

In [422]:
from sqlalchemy import create_engine

import pymysql
#connect
conn=pymysql.connect(host='localhost',user='root')
cursor = conn.cursor()
print("Connected to mySQL!")

# create database

DB_NAME = 'Baseball'
cursor.execute('DROP DATABASE IF EXISTS %s' %DB_NAME)
cursor.execute('CREATE DATABASE IF NOT EXISTS %s' %DB_NAME)




Connected to mySQL!


1

In [423]:
tableName   = "Pitcher_info"

sqlEngine       = create_engine('mysql+pymysql://root:@127.0.0.1/Baseball', pool_recycle=3600)

dbConnection    = sqlEngine.connect()

 

try:

    frame           = pitcher_info.to_sql(tableName, dbConnection, if_exists='replace');

except ValueError as vx:

    print(vx)

except Exception as ex:   

    print(ex)

else:

    print("Table %s created successfully."%tableName);   

finally:

    dbConnection.close()

Table Pitcher_info created successfully.
